## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [32]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [45]:
# local path to downloaded results
iteration = '_08' # <----- options: 'base', '_01'~'_06'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}/baseline' 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_08/baseline


In [46]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,30,2020-11-07 01:36:44,2020-11-07 01:38:04,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,8.220871,2.976256,2.245580,1.766323,16.272727,18.363636,14.853651,12.460630,5250.0,17.966103
1,2,25,2020-11-07 01:01:35,2020-11-07 01:03:11,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,21.361470,9.455010,6.586602,4.926889,15.363636,16.909091,49.236152,25.884132,4528.0,60.709783
2,3,23,2020-11-07 02:59:19,2020-11-07 03:00:51,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.255550,1.709567,1.896317,1.179670,15.636364,18.000000,13.834989,6.485464,4528.0,17.973358
3,4,16,2020-11-07 00:58:10,2020-11-07 01:16:03,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,115.761329,34.681788,24.499500,17.837186,15.909091,17.000000,318.065514,248.749716,5079.0,366.385557
4,5,3,2020-11-06 23:05:15,2020-11-06 23:06:44,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,4.994340,2.041792,1.460034,1.475423,17.636364,18.636364,8.730088,5.653697,5154.0,9.582572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,9,2020-11-06 20:05:54,2020-11-06 20:07:39,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.891203,1.142548,2.045235,1.169018,12.909091,17.545455,15.984136,6.966411,5150.0,17.823517
39996,39997,36,2020-11-07 03:25:11,2020-11-07 03:26:40,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,5.670990,2.090462,1.562282,1.502567,15.727273,18.818182,12.545181,5.662886,4528.0,16.059910
39997,39998,30,2020-11-07 03:50:58,2020-11-07 04:22:25,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,272.613472,62.180450,27.133279,33.341630,15.545455,17.363636,453.181858,379.923450,3326.0,612.630815
39998,39999,13,2020-11-07 04:07:30,2020-11-07 04:08:55,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,7.006728,2.214624,1.380167,1.149487,16.363636,16.909091,10.741592,8.539463,4503.0,12.303307


In [47]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'build_existing_model.hvac_system_cooling_type'
else:
    cooling_col = 'build_existing_model.hvac_cooling_type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'build_existing_model.hvac_system_heating_natural_gas'
else:
    heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')


selected


### Check housing charateristics distributions

In [48]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_08/baseline - RESULT SUMMARY:

  * 39946 / 40000 samples ran successfully, 54 failed, efficacy: 99.9% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1            0.503380
2            0.303259
3            0.193361
N_failed    54.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry      0.699319
WoodStud     0.300681
N_failed    54.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59      0.235167
1960-79      0.255094
1980-99      0.137761
2000-09      0.076203
2010s        0.011691
<1940        0.284084
N_failed    54.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC     0.616958
Heat Pump      0.025635
None           0.052421
Room AC        0.304987
N_failed      54.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP        

### Add additional metrics to ResStock results

In [49]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['build_existing_model.geometry_floor_area'].map(sqft_map)
res['gas_eui'] = res['simulation_output_report.total_site_natural_gas_therm'].divide(res['sqft'])
res['elec_eui'] = res['simulation_output_report.total_site_electricity_kwh'].divide(res['sqft'])


# (2) aggregate heating and cooling end uses in ResStock results
res['total_gas_heating'] = res['simulation_output_report.natural_gas_central_system_heating_therm'] +\
    res['simulation_output_report.natural_gas_heating_therm']

res['total_elec_heating'] = res['simulation_output_report.electricity_central_system_heating_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_heating_kwh'] +\
    res['simulation_output_report.electricity_fans_heating_kwh'] +\
    res['simulation_output_report.electricity_heating_kwh'] +\
    res['simulation_output_report.electricity_pumps_heating_kwh']

res['total_elec_cooling'] = res['simulation_output_report.electricity_central_system_cooling_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_cooling_kwh'] +\
    res['simulation_output_report.electricity_cooling_kwh'] +\
    res['simulation_output_report.electricity_fans_cooling_kwh'] +\
    res['simulation_output_report.electricity_pumps_cooling_kwh']

print('new cols added to "res"')

new cols added to "res"


In [73]:
### get sqft distr by building type and vintage
res_sqft_distr = res.groupby(['build_existing_model.geometry_building_type_recs', 'build_existing_model.vintage_acs',
    'build_existing_model.geometry_floor_area'])['building_id'].count().to_frame()

# normalize
res_sqft_distr = res_sqft_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)

# export
res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_floor_area.csv'), index=True)
print('sqft distr by vintage & bldg type exported')


build_existing_model.geometry_floor_area                                              0-499  \
build_existing_model.geometry_building_type_recs build_existing_model.vintage_acs             
Mobile Home                                      1940-59                                NaN   
                                                 1960-79                                NaN   
                                                 1980-99                                NaN   
                                                 2000-09                           0.018519   
                                                 2010s                                  NaN   
                                                 <1940                                  NaN   
Multi-Family with 2 - 4 Units                    1940-59                           0.085075   
                                                 1960-79                           0.255652   
                                                 1

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [50]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [51]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    return res_group_i

Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN

### (1) get median consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i[res_group_i['completed_status']=="Success"])
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> median consumption computed')

### (2) get mean consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> mean consumption computed')


>> median consumption computed
>> mean consumption computed


In [52]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui', 
               'median elec eui': 'elec_eui'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui', 
               'mean elec eui': 'elec_eui'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:')

groups[['HousingGroupNo','HousingGroupName','median gas','mean gas','median gas eui', 'mean gas eui',
        'median elec','mean elec', 'median elec eui','mean elec eui']]

>> median & mean sqft & eui computed:


,HousingGroupNo,HousingGroupName,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
0,1,Masonry Post-1978 less than 2 stories,621.315067,666.999464,0.473084,0.488893,7942.045802,9011.961673,5.856340,6.628325
1,2,Masonry All Years Split Level,1157.556298,1452.847781,0.623358,0.683112,10105.446177,11987.415629,5.277901,6.175320
2,3,Masonry Post-1978 2 or more stories,990.113438,1025.888691,0.449113,0.506282,9676.351413,11150.583860,4.412536,5.278951
3,4,Masonry 1942-1978 Less than 2 Stories,1011.180278,1206.988178,0.595484,0.636993,9526.883618,11078.504369,5.356641,6.190340
4,5,Masonry 1942-1978 2 or more stories,1727.803472,2271.677914,0.603110,0.695961,12675.553379,15367.809040,4.280036,4.990233
5,6,Masonry Pre-1942 less than 2 stories,946.445631,1053.817267,0.839444,0.888299,8900.436466,9822.193240,7.408271,8.552945
6,7,Masonry Pre-1942 2 or more stories,1379.069677,1441.734815,0.896971,0.917161,9999.737383,11310.291168,6.402217,7.305889
7,8,Frame Post-1978 Less than 2 stories,654.430061,707.953709,0.507477,0.538934,7957.320966,8788.335264,5.959327,6.675531
8,9,Frame Post-1978 Split Level,753.647896,818.938523,0.508102,0.545398,8454.969206,9544.750181,5.648010,6.383668
9,10,Frame Post-1978 2 or more stories,1030.045221,1089.482407,0.508683,0.561154,9592.819611,11388.636942,4.789589,5.672204


In [53]:
### (4) get heating and cooling end uses to "groups"
Metric_map1 = ['frac_gas_heating', 'frac_elec_heating', 'frac_elec_cooling']
Metric_map2 = {'size_heating_kbut_h': 'simulation_output_report.size_heating_system_kbtu_h',
               'size_cooling_kbut_h': 'simulation_output_report.size_cooling_system_kbtu_h'}

## initialize
for metric in Metric_map1 + list(Metric_map2.keys()):
    groups[metric] = np.NAN

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (4.1) get fractions
    groups.iloc[i, groups.columns.get_loc('frac_gas_heating')] = \
        res_group_i['total_gas_heating'].mean()/res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()

    groups.iloc[i, groups.columns.get_loc('frac_elec_heating')] = \
        res_group_i['total_elec_heating'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('frac_elec_cooling')] = \
        res_group_i['total_elec_cooling'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    ### (4.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('computed')

computed


In [54]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'build_existing_model.geometry_floor_area',
              'build_existing_model.insulation_unfinished_attic']

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i.query('completed_status=="Success"'))
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['building_id'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [55]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   861.0   
2                                  1416                       9661  8946.0   
3                                  1812                      18225   382.0   
4                                  1210                       8779  3559.0   
5                                  1427                      10549  2481.0   
6                                  1504                       9299   769.0   
7                                  2044                      13272   894.0   
8                                  1054                       9353   919.0   
9                                  1168                      10445  1296.0   
10                                 1422                      13495   377.0   
11                                 1176                       9213  2815.0   
12                                 1297                       9964  2096.0   
13                                 1530                      12543  1572.0   
14                                 1340                       9103  1015.0   
15                                 1753                      11967  1081.0   

                                                                      \
                 median gas   median elec     mean gas     mean elec   
HousingGroupNo                                                         
1                621.315067   7942.045802   666.999464   9011.961673   
2               1157.556298  10105.446177  1452.847781  11987.415629   
3                990.113438   9676.351413  1025.888691  11150.583860   
4               1011.180278   9526.883618  1206.988178  11078.504369   
5               1727.803472  12675.553379  2271.677914  15367.809040   
6                946.445631   8900.436466  1053.817267   9822.193240   
7               1379.069677   9999.737383  1441.734815  11310.291168   
8                654.430061   7957.320966   707.953709   8788.335264   
9                753.647896   8454.969206   818.938523   9544.750181   
10              1030.045221   9592.819611  1089.482407  11388.636942   
11               979.740707   9254.631021  1162.742779  10593.269694   
12              1175.673633   9216.776540  1235.083587  10606.616985   
13              1632.101564  12346.142751  2134.787905  15174.709758   
14               968.123790   8630.761394  1029.210801   9689.759461   
15              1388.141091   9929.690771  1428.386898  11467.496160   

                                                 ...  \
               Gas Diff-med(%) Elec Diff-med(%)  ...   
HousingGroupNo                                   ...   
1                    45.161954        22.757773  ...   
2                    18.251674        -4.600416  ...   
3                    45.357978        46

### Write output to folder

In [56]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_08/baseline
